In [1]:
import pandas as pd
import pyodbc
import numpy as np

In [2]:
# Import data from SQL Server database (based on orig currency)
sql_conn = pyodbc.connect('DRIVER={SQL Server};\
                            SERVER=azscia01.amr.corp.intel.com,1433;\
                            DATABASE=CDM;\
                            Trusted_Connection=yes;\
                            integrated security=true')
query= """WITH Cntrct as
(
SELECT T.[DocumentId]
      ,T.[TableNumber]
      ,T.[RowNumber]
      ,T.[ColumnNumber]
      ,T.[ColumnValue]
      ,T.[TableConfidence]
  FROM [CDM].[dbo].[tablesunpivoted] AS T
  JOIN [dbo].[draftpublishedcontracts] AS C
  ON C.[Document Id] = T.DocumentId
  WHERE C.[Doc Status] IN ('Published','Draft')
)

SELECT
Cntrct.DocumentId
,Cntrct.TableNumber
,Cntrct.ColumnNumber
,Cntrct.ColumnValue
,CASE WHEN case when ISNUMERIC(RTRIM(LTRIM(replace(replace(Cntrct.ColumnValue,'$',''),',',''))))=1 then TRY_CONVERT(decimal(38,2),RTRIM(LTRIM(replace(replace(Cntrct.ColumnValue,'$',''),',','')))) else null end = TRY_CONVERT(decimal(38,2),CntrctSysMerge.line_net_prc_amt) THEN 'Price' ELSE 'Not Price' END as Label
FROM Cntrct

LEFT JOIN 
(
SELECT *
FROM (
	SELECT distinct itm_id,line_net_prc_amt
	FROM [dbo].[OutlineAgreement]
	UNION
	SELECT distinct [itm_id],[new_buy_prc_amt]
	FROM [dbo].[WIINGS]
) AS Systm
JOIN Cntrct
ON (Cntrct.ColumnValue = Systm.itm_id)
) AS CntrctSysMerge

ON (Cntrct.DocumentId = CntrctSysMerge.DocumentId and Cntrct.TableNumber = CntrctSysMerge.TableNumber and Cntrct.RowNumber = CntrctSysMerge.RowNumber)"""
df_Training = pd.read_sql(query, sql_conn)
sql_conn.close()

In [ ]:
# Import data from SQL Server database (based on spn)
sql_conn = pyodbc.connect('DRIVER={SQL Server};\
                            SERVER=azscia01.amr.corp.intel.com,1433;\
                            DATABASE=CDM;\
                            Trusted_Connection=yes;\
                            integrated security=true')
query= """WITH Cntrct as
(
SELECT T.[DocumentId]
      ,T.[TableNumber]
      ,T.[RowNumber]
      ,T.[ColumnNumber]
      ,T.[ColumnValue]
      ,T.[TableConfidence]
	  ,C.[Supplier Id]
  FROM [CDM].[dbo].[tablesunpivoted] AS T
  JOIN [dbo].[draftpublishedcontracts] AS C
  ON C.[Document Id] = T.DocumentId
  WHERE C.[Doc Status] IN ('Published','Draft')
)

SELECT
Cntrct.DocumentId
,Cntrct.TableNumber
,Cntrct.ColumnNumber
,Cntrct.ColumnValue
,CASE WHEN case when ISNUMERIC(RTRIM(LTRIM(replace(replace(Cntrct.ColumnValue,'$',''),',',''))))=1 then TRY_CONVERT(decimal(38,2),RTRIM(LTRIM(replace(replace(Cntrct.ColumnValue,'$',''),',','')))) else null end = TRY_CONVERT(decimal(38,2),CntrctSysMerge.line_net_prc_amt) THEN 'Price' ELSE 'Not Price' END as Label
FROM Cntrct

LEFT JOIN 
(
SELECT *
FROM (
	SELECT distinct [supl_id],[supl_mtrl_id] as spn,line_net_prc_amt
	FROM [dbo].[OutlineAgreement]
	where [supl_mtrl_id] is not null
	UNION
	SELECT distinct [curr_supl_id],[curr_supl_itm_nbr],[new_buy_prc_amt]
	FROM [dbo].[WIINGS]
	where [curr_supl_itm_nbr] is not null
) AS Systm
JOIN Cntrct
ON (Cntrct.ColumnValue = Systm.spn)
) AS CntrctSysMerge

ON (Cntrct.DocumentId = CntrctSysMerge.DocumentId and Cntrct.TableNumber = CntrctSysMerge.TableNumber and Cntrct.RowNumber = CntrctSysMerge.RowNumber)"""
df_Training = pd.read_sql(query, sql_conn)
sql_conn.close()

In [3]:
df_filtered = df_Training[df_Training['Label'] == 'Price']
df_grouped = df_filtered.groupby(['DocumentId','TableNumber','ColumnNumber'])['ColumnNumber'].count().reset_index(name="count")
df_grouped

,DocumentId,TableNumber,ColumnNumber,count
0,Doc1003512767,3,3,1
1,Doc1003512767,3,4,9
2,Doc1005740068,3,2,1
3,Doc1008326585,46,2,11
4,Doc1008326585,47,2,3
...,...,...,...,...
1925,Doc977436468,6,7,2
1926,Doc979301475,6,7,2
1927,Doc981223975,7,7,2
1928,Doc996552509,0,3,1


In [4]:
df_grouped = df_grouped.loc[df_grouped.groupby(["DocumentId", "TableNumber"])["count"].idxmax()]
df_grouped['DocNumTableNum'] = df_grouped['DocumentId'].apply(str) + '/' + df_grouped['TableNumber'].apply(str)
df_grouped['DocNumTableNumColumnNumber'] = df_grouped['DocumentId'].apply(str) + '/' + df_grouped['TableNumber'].apply(str) + '/' + df_grouped['ColumnNumber'].apply(str)
df_grouped

,DocumentId,TableNumber,ColumnNumber,count,DocNumTableNum,DocNumTableNumColumnNumber
1,Doc1003512767,3,4,9,Doc1003512767/3,Doc1003512767/3/4
2,Doc1005740068,3,2,1,Doc1005740068/3,Doc1005740068/3/2
3,Doc1008326585,46,2,11,Doc1008326585/46,Doc1008326585/46/2
4,Doc1008326585,47,2,3,Doc1008326585/47,Doc1008326585/47/2
5,Doc1008326585,48,3,2,Doc1008326585/48,Doc1008326585/48/3
...,...,...,...,...,...,...
1921,Doc975084460,2,16,47,Doc975084460/2,Doc975084460/2/16
1925,Doc977436468,6,7,2,Doc977436468/6,Doc977436468/6/7
1926,Doc979301475,6,7,2,Doc979301475/6,Doc979301475/6/7
1927,Doc981223975,7,7,2,Doc981223975/7,Doc981223975/7/7


In [5]:
df_t = df_Training.groupby(['DocumentId','TableNumber','ColumnNumber']).ColumnValue.unique().reset_index()
df_t

,DocumentId,TableNumber,ColumnNumber,ColumnValue
0,Doc1000015244,0,0,"[BY:, NAME:, TITLE:]"
1,Doc1000015244,0,1,"[BY:, NAME:, TITLE:]"
2,Doc1000015244,1,0,"[Equipment\nModel, Synergis MX, Notes]"
3,Doc1000015244,1,1,"[Material\nMaster\nNumber, TBD, - Ozone Genera..."
4,Doc1000015244,1,2,"[Material\nMaster\nDescription, TBD]"
...,...,...,...,...
772303,Doc999948248,15,3,"[, Yes, Hardware Costs, yes]"
772304,Doc999948248,16,0,"[German Language Pack, Spanish Language Pack, ..."
772305,Doc999948248,16,1,"[456-101-188, 456-101-190, 456-101-189, 457-10..."
772306,Doc999948248,16,2,"[Per Discovered\nCustodian, Per Reviewed\nCust..."


In [6]:
df_t['DocNumTableNum'] = df_t['DocumentId'] + '/' + df_t['TableNumber'].apply(str)
df_t['DocNumTableNumColumnNumber'] = df_t['DocumentId'] + '/' + df_t['TableNumber'].apply(str) + '/' + df_t['ColumnNumber'].apply(str)
df_t['Label'] = np.where(df_t['DocNumTableNumColumnNumber'].isin(df_grouped.DocNumTableNumColumnNumber.tolist()), 'Price', 'Other')
df_t['PriceInTable'] = np.where(df_t['DocNumTableNum'].isin(df_grouped.DocNumTableNum.tolist()), 'Yes','No')
df_t['Label'] = df_t.apply(lambda row: 'NotPrice' if (row['Label']=='Other')&(row['PriceInTable'] == 'Yes' ) else row['Label'],axis=1)
#df_t[df_t['Label']=='Other']
#df_t[(df_t['Label']=='Other')&(df_t['test']=='NotPrice')]
df_t

,DocumentId,TableNumber,ColumnNumber,ColumnValue,DocNumTableNum,DocNumTableNumColumnNumber,Label,PriceInTable
0,Doc1000015244,0,0,"[BY:, NAME:, TITLE:]",Doc1000015244/0,Doc1000015244/0/0,Other,No
1,Doc1000015244,0,1,"[BY:, NAME:, TITLE:]",Doc1000015244/0,Doc1000015244/0/1,Other,No
2,Doc1000015244,1,0,"[Equipment\nModel, Synergis MX, Notes]",Doc1000015244/1,Doc1000015244/1/0,Other,No
3,Doc1000015244,1,1,"[Material\nMaster\nNumber, TBD, - Ozone Genera...",Doc1000015244/1,Doc1000015244/1/1,Other,No
4,Doc1000015244,1,2,"[Material\nMaster\nDescription, TBD]",Doc1000015244/1,Doc1000015244/1/2,Other,No
...,...,...,...,...,...,...,...,...
772303,Doc999948248,15,3,"[, Yes, Hardware Costs, yes]",Doc999948248/15,Doc999948248/15/3,Other,No
772304,Doc999948248,16,0,"[German Language Pack, Spanish Language Pack, ...",Doc999948248/16,Doc999948248/16/0,Other,No
772305,Doc999948248,16,1,"[456-101-188, 456-101-190, 456-101-189, 457-10...",Doc999948248/16,Doc999948248/16/1,Other,No
772306,Doc999948248,16,2,"[Per Discovered\nCustodian, Per Reviewed\nCust...",Doc999948248/16,Doc999948248/16/2,Other,No


In [7]:
import re
import nltk
#nltk.download('stopwords')

def listToString(s):  
    str1 = " "  
    return str1.join(s)

def remove_numbers(wordtoken):
    return re.sub(r'\d',r'_',wordtoken)

def tokenize_string(strng):
    #if not strng:
    #    print('The text to be tokenized is a None type. Defaulting to blank string.')
    #    strng = ''
    return nltk.word_tokenize(strng)

punct= list('"\'()*`{|}~/')
def remove_punct(wordtokens):
    return [wordtoken for wordtoken in wordtokens if wordtoken not in punct]

df_t['Col_Tokens'] = df_t.ColumnValue.apply(listToString)
df_t['Col_Tokens'] = df_t.Col_Tokens.apply(remove_numbers)
df_t['Col_Tokens'] = df_t.Col_Tokens.apply(tokenize_string)

In [8]:
stopwords= set(nltk.corpus.stopwords.words('english'))
def remove_stopwords(wordtokens):
    return [wordtoken for wordtoken in wordtokens if wordtoken not in stopwords]

# Porter Stem Tokens
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
def stem_tokens(wordtokens):
    return [ps.stem(word) for word in wordtokens]

df_t['Col_Tokens'] = df_t.Col_Tokens.apply(remove_stopwords)
df_t['Col_Tokens'] = df_t.Col_Tokens.apply(remove_punct)
df_t['Col_Tokens'] = df_t.Col_Tokens.apply(stem_tokens)

In [9]:
df_tr = df_t[(df_t['Label']=='Price') | (df_t['Label']=='NotPrice')]
df_tr[(df_tr['Label']=='Price')]

,DocumentId,TableNumber,ColumnNumber,ColumnValue,DocNumTableNum,DocNumTableNumColumnNumber,Label,PriceInTable,Col_Tokens
1686,Doc1003512767,3,4,"[Price\n(US$/pc), 119.42, 114.42, 153.00, for ...",Doc1003512767/3,Doc1003512767/3/4,Price,Yes,"[price, US, $, /pc, ___.__, ___.__, ___.__, de..."
3423,Doc1005740068,3,2,"[Price, $31.00, $1.09, $1.07]",Doc1005740068/3,Doc1005740068/3/2,Price,Yes,"[price, $, __.__, $, _.__, $, _.__]"
4929,Doc1008326585,46,2,"[Price/Unit, $ 4,289.25, $ 1,500.00, Intel Dir...",Doc1008326585/46,Doc1008326585/46/2,Price,Yes,"[price/unit, $, _, ,, ___.__, $, _, ,, ___.__,..."
4934,Doc1008326585,47,2,"[Price/Unit, $ 6,943.05, Intel Direct Source, ...",Doc1008326585/47,Doc1008326585/47/2,Price,Yes,"[price/unit, $, _, ,, ___.__, intel, direct, s..."
4942,Doc1008326585,48,3,"[IPN, 6,943.05, 6,536.00, 500241007, 500282670...",Doc1008326585/48,Doc1008326585/48/3,Price,Yes,"[ipn, _, ,, ___.__, _, ,, ___.__, _________, _..."
...,...,...,...,...,...,...,...,...,...
758867,Doc975084460,2,16,"[PRICE\n$/UM\nChina to\nTMM, 10.20, 17.55, , 8...",Doc975084460/2,Doc975084460/2/16,Price,Yes,"[price, $, /um, china, tmm, __.__, __.__, __._..."
759973,Doc977436468,6,7,"[Unit Price (US$), $ 2.250, $\n2.250]",Doc977436468/6,Doc977436468/6/7,Price,Yes,"[unit, price, US, $, $, _.___, $, _.___]"
760519,Doc979301475,6,7,"[Unit Price (US$), $ 2.250, $\n2.250]",Doc979301475/6,Doc979301475/6/7,Price,Yes,"[unit, price, US, $, $, _.___, $, _.___]"
761214,Doc981223975,7,7,"[Unit Price (US$), $ 2.250, $\n2.250]",Doc981223975/7,Doc981223975/7/7,Price,Yes,"[unit, price, US, $, $, _.___, $, _.___]"


In [10]:
df_tr_final = df_t[df_t['DocNumTableNum'].isin(df_tr.DocNumTableNum.tolist())]
df_tr_final

,DocumentId,TableNumber,ColumnNumber,ColumnValue,DocNumTableNum,DocNumTableNumColumnNumber,Label,PriceInTable,Col_Tokens
1682,Doc1003512767,3,0,"[Product, RHB66P, RHB66P-MA, RHB66P-MAH, IND22...",Doc1003512767/3,Doc1003512767/3/0,NotPrice,Yes,"[product, rhb__p, rhb__p-ma, rhb__p-mah, ind__..."
1683,Doc1003512767,3,1,"[Size\n(mm), cp762, tp762, (p762, (p742, cp742...",Doc1003512767/3,Doc1003512767/3/1,NotPrice,Yes,"[size, mm, cp___, tp___, p___, p___, cp___, cp..."
1684,Doc1003512767,3,2,"[IPN, 500181753, 500181869, 500181870, 5001818...",Doc1003512767/3,Doc1003512767/3/2,NotPrice,Yes,"[ipn, _________, _________, _________, _______..."
1685,Doc1003512767,3,3,"[Qty, , pc, 145.00, MOQ: 20 pcs, 20 pcs - 50 p...",Doc1003512767/3,Doc1003512767/3/3,NotPrice,Yes,"[qti, pc, ___.__, moq, :, __, pc, __, pc, -, _..."
1686,Doc1003512767,3,4,"[Price\n(US$/pc), 119.42, 114.42, 153.00, for ...",Doc1003512767/3,Doc1003512767/3/4,Price,Yes,"[price, US, $, /pc, ___.__, ___.__, ___.__, de..."
...,...,...,...,...,...,...,...,...,...
771117,Doc996552509,0,3,"[Qty, , pc, 145.00, MOQ: 20 pcs, 20 pcs - 50 p...",Doc996552509/0,Doc996552509/0/3,NotPrice,Yes,"[qti, pc, ___.__, moq, :, __, pc, __, pc, -, _..."
771118,Doc996552509,0,4,"[Price\n(USS/pc), 119.42, 114.42, 153.00, for ...",Doc996552509/0,Doc996552509/0/4,Price,Yes,"[price, uss/pc, ___.__, ___.__, ___.__, delive..."
771119,Doc996552509,0,5,"[Incoterms, , FOB Marubeni's\nwarehouse nearby...",Doc996552509/0,Doc996552509/0/5,NotPrice,Yes,"[incoterm, fob, marubeni', warehous, nearbi, s..."
771120,Doc996552509,0,6,"[Pad Shelf Life, , 9 months from\nmanufacturin...",Doc996552509/0,Doc996552509/0/6,NotPrice,Yes,"[pad, shelf, life, _, month, manufactur, date,..."


In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_tr_final.loc[:, 'Col_Tokens'], df_tr_final.loc[:, 'Label'], test_size=0.3, random_state=1)

In [12]:
def dummy_analyzer(x):
    return x

from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
rf_pipeline = Pipeline([
    ('bow', CountVectorizer(analyzer=dummy_analyzer)),
    ('RandomForestClassifier', RandomForestClassifier(n_estimators=100))
])

rf_pipeline.fit(X_train, y_train)
rf_predictions= rf_pipeline.predict(X_test)

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
print(classification_report(y_test, rf_predictions))
print(confusion_matrix(y_test, rf_predictions))

              precision    recall  f1-score   support

    NotPrice       0.99      0.99      0.99      3097
       Price       0.93      0.89      0.91       377

    accuracy                           0.98      3474
   macro avg       0.96      0.94      0.95      3474
weighted avg       0.98      0.98      0.98      3474

[[3073   24]
 [  40  337]]


In [13]:
df_t['Prediction'] = rf_pipeline.predict(df_t['Col_Tokens'])
df_t['Probability'] = np.amax(rf_pipeline.predict_proba(df_t['Col_Tokens']),1)

In [14]:
df_t[df_t['Label']=='Price']
#df_t[((df_t['Prediction']=='Price')&(df_t['Label']=='Other'))]

,DocumentId,TableNumber,ColumnNumber,ColumnValue,DocNumTableNum,DocNumTableNumColumnNumber,Label,PriceInTable,Col_Tokens,Prediction,Probability
1686,Doc1003512767,3,4,"[Price\n(US$/pc), 119.42, 114.42, 153.00, for ...",Doc1003512767/3,Doc1003512767/3/4,Price,Yes,"[price, US, $, /pc, ___.__, ___.__, ___.__, de...",Price,0.51
3423,Doc1005740068,3,2,"[Price, $31.00, $1.09, $1.07]",Doc1005740068/3,Doc1005740068/3/2,Price,Yes,"[price, $, __.__, $, _.__, $, _.__]",Price,1.00
4929,Doc1008326585,46,2,"[Price/Unit, $ 4,289.25, $ 1,500.00, Intel Dir...",Doc1008326585/46,Doc1008326585/46/2,Price,Yes,"[price/unit, $, _, ,, ___.__, $, _, ,, ___.__,...",Price,1.00
4934,Doc1008326585,47,2,"[Price/Unit, $ 6,943.05, Intel Direct Source, ...",Doc1008326585/47,Doc1008326585/47/2,Price,Yes,"[price/unit, $, _, ,, ___.__, intel, direct, s...",Price,0.98
4942,Doc1008326585,48,3,"[IPN, 6,943.05, 6,536.00, 500241007, 500282670...",Doc1008326585/48,Doc1008326585/48/3,Price,Yes,"[ipn, _, ,, ___.__, _, ,, ___.__, _________, _...",Price,0.90
...,...,...,...,...,...,...,...,...,...,...,...
758867,Doc975084460,2,16,"[PRICE\n$/UM\nChina to\nTMM, 10.20, 17.55, , 8...",Doc975084460/2,Doc975084460/2/16,Price,Yes,"[price, $, /um, china, tmm, __.__, __.__, __._...",Price,0.86
759973,Doc977436468,6,7,"[Unit Price (US$), $ 2.250, $\n2.250]",Doc977436468/6,Doc977436468/6/7,Price,Yes,"[unit, price, US, $, $, _.___, $, _.___]",Price,0.85
760519,Doc979301475,6,7,"[Unit Price (US$), $ 2.250, $\n2.250]",Doc979301475/6,Doc979301475/6/7,Price,Yes,"[unit, price, US, $, $, _.___, $, _.___]",Price,0.85
761214,Doc981223975,7,7,"[Unit Price (US$), $ 2.250, $\n2.250]",Doc981223975/7,Doc981223975/7/7,Price,Yes,"[unit, price, US, $, $, _.___, $, _.___]",Price,0.85


In [18]:
df_t[['DocumentId','TableNumber','ColumnNumber','Label','Prediction','Probability']].to_csv(r'C:\Users\jrstrong\Desktop\temp\PriceColumns_Short.csv', index=False)

In [15]:
# this is really really slow
sql_conn = pyodbc.connect('DRIVER={SQL Server};\
                            SERVER=sql1944-fm1-in.amr.corp.intel.com,3181;\
                            DATABASE=GSMDW_TST;\
                            Trusted_Connection=yes;\
                            integrated security=true')

cursor = sql_conn.cursor()
cursor.execute('TRUNCATE TABLE [GSMDW_TST].[JUSTIN].[PriceColumns]')

#cursor.fast_executemany = True

for index,row in df_t.iterrows():
    cursor.execute('INSERT INTO [GSMDW_TST].[JUSTIN].[PriceColumns]([DocNum],[TableNum],[ColumnNumber],[Label],[Prediction],[Probability]) VALUES (?,?,?,?,?,?)', row[0],row[1],row[2],row[6],row[9],row[10])
    #cursor.executemany('INSERT INTO [GSMDW_TST].[JUSTIN].[PriceColumns]([DocNum],[TableNum],[ColumnNumber],[Label],[Prediction],[Probability]) VALUES (?,?,?,?,?,?)', row[0],row[1],row[2],row[6],row[9],row[10])
    sql_conn.commit()

sql_conn.commit()

sql_conn.close()

TypeError: function takes exactly 2 arguments (7 given)